# Azure Cognitive Search LangChain Vector Code Sample
This code demonstrates how to use Azure Cognitive Search with OpenAI and the Azure Cognitive Search LangChain Vector Store
To run the code, install the following packages. Please use the latest pre-release version `pip install azure-search-documents --pre`.

In [ ]:
! pip install azure-search-documents --pre 
! pip install openai
! pip install python-dotenv
! pip install langchain

## Import required libraries and environment variables

In [1]:
# Import required libraries  
import openai
import os  
from dotenv import load_dotenv
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch
from azure.search.documents.indexes.models import (
    SemanticSettings,
    SemanticConfiguration,
    PrioritizedFields,
    SemanticField
)


## Configure OpenAI Settings

In [2]:
# Configure environment variables  
load_dotenv()  
openai.api_type: str = "azure"  
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")  
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")  
openai.api_version = os.getenv("AZURE_OPENAI_API_VERSION")  
model: str = "text-embedding-ada-002"

## Configure Vector Store Settings

In [3]:
vector_store_address: str = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")  
vector_store_password: str = os.getenv("AZURE_SEARCH_ADMIN_KEY") 
index_name: str = "shadow-vector-search-langchain" 

## Create embeddings and vector store instances
Read your data, generate OpenAI embeddings and export to a format to insert your Azure Cognitive Search index:

In [5]:
embeddings: OpenAIEmbeddings = OpenAIEmbeddings(deployment=model, model=model, chunk_size=1, openai_api_key=openai.api_key, openai_api_base=os.getenv("AZURE_OPENAI_ENDPOINT"), openai_api_type="azure" )
#index_name: str = "langchain-vector-demo"
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
    semantic_configuration_name='my-semantic-config-langchain',
        semantic_settings=SemanticSettings(
            default_configuration='my-semantic-config-langchain',
            configurations=[
                SemanticConfiguration(
                    name='my-semantic-config-langchain',
                    prioritized_fields=PrioritizedFields(
                        title_field=SemanticField(field_name='content'),
                        prioritized_content_fields=[SemanticField(field_name='content')],
                        prioritized_keywords_fields=[SemanticField(field_name='metadata')] # by default when data is loaded metadata contains the source directory frm where the document was loaded
                    ))
            ])
    )


## Insert text and embeddings into vector store
Load text files into the index

In [ ]:
import tiktoken
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

# open and read all the txt files and put them into the index

directory = "../data/txt/challenger_customer/"

for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        loader = TextLoader(os.path.join(directory, filename), encoding="utf-8")
        documents = loader.load()
        text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
        docs = text_splitter.split_documents(documents)

        vector_store.add_documents(documents=docs)


## Perform a vector similarity search

In [10]:
# Perform a similarity search
docs = vector_store.similarity_search(
    query="What is challenger sales model",
    k=3,
    search_type="similarity",
)
results = [doc.page_content for doc in docs]
print("\n".join(results))

Chapter 10. Shifting to a Challenger commercial model - implications and implementation lessons.  To win today, you need a challenger inside the customer organization. That is the central premise of this entire book. It turns out the far bigger story isn't about suppliers struggle to sell solutions, It's the customers struggle to buy them. Arming mobilizers with world class commercial insights and supporting their efforts to rally consensus for that insight and inherently then for your solution, requires a new go to market strategy. The implications of such a significant shift in commercial strategy are numerous. Sales and marketing must work together, bound by a new common language of disruption. Seller skills, something we covered in depth in the Challenger sale, must be reconsidered. Marketing content must be atomized and lead customers to a clear narrative that discloses what they've gotten wrong. There are likely sales coverage and territory implications for many organizations as 

## Perform a hybrid search

In [13]:
# Perform a hybrid search
docs = vector_store.hybrid_search(
    query="What is challenger sales model",
    k=3, 
    search_type="hybrid"
)
results = [doc.page_content for doc in docs]
print("\n".join(results))

Chapter 10. Shifting to a Challenger commercial model - implications and implementation lessons.  To win today, you need a challenger inside the customer organization. That is the central premise of this entire book. It turns out the far bigger story isn't about suppliers struggle to sell solutions, It's the customers struggle to buy them. Arming mobilizers with world class commercial insights and supporting their efforts to rally consensus for that insight and inherently then for your solution, requires a new go to market strategy. The implications of such a significant shift in commercial strategy are numerous. Sales and marketing must work together, bound by a new common language of disruption. Seller skills, something we covered in depth in the Challenger sale, must be reconsidered. Marketing content must be atomized and lead customers to a clear narrative that discloses what they've gotten wrong. There are likely sales coverage and territory implications for many organizations as 

## Perform a Hybrid Search with Semantic re-ranking (powered by Bing)

In [12]:
# Perform a hybrid search with semantic reranking  
docs_and_scores = vector_store.semantic_hybrid_search_with_score(  
    query="What is challenger sales model",  
    k=3,  
)  
  
# Print the results  
for doc, score in docs_and_scores:  
    print("-" * 80)  
    answers = doc.metadata['answers']  
    if answers:  
        if answers.get('highlights'):  
            print(f"Semantic Answer: {answers['highlights']}")  
        else:  
            print(f"Semantic Answer: {answers['text']}")  
        print(f"Semantic Answer Score: {score}")  
    print("Content:", doc.page_content)  
    captions = doc.metadata['captions']
    print(f"Score: {score}") 
    if captions:  
        if captions.get('highlights'):  
            print(f"Caption: {captions['highlights']}")  
        else:  
            print(f"Caption: {captions['text']}")  
    else:  
        print("Caption not available")  


--------------------------------------------------------------------------------
Content: Chapter 10. Shifting to a Challenger commercial model - implications and implementation lessons.  To win today, you need a challenger inside the customer organization. That is the central premise of this entire book. It turns out the far bigger story isn't about suppliers struggle to sell solutions, It's the customers struggle to buy them. Arming mobilizers with world class commercial insights and supporting their efforts to rally consensus for that insight and inherently then for your solution, requires a new go to market strategy. The implications of such a significant shift in commercial strategy are numerous. Sales and marketing must work together, bound by a new common language of disruption. Seller skills, something we covered in depth in the Challenger sale, must be reconsidered. Marketing content must be atomized and lead customers to a clear narrative that discloses what they've gotten wr